In [ ]:
import torch
import numpy as np
import os
from torch.utils.data import Subset
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader
from torch_geometric.data import Batch
import random
from matplotlib import pyplot as plt
from utils import set_seed

from data import MIDIDataset, graph_from_tensor, graph_from_tensor_torch
from model import VAE
from utils import plot_struct, dense_from_sparse, dense_from_sparse_torch, muspy_from_dense, muspy_from_dense_torch
from utils import plot_pianoroll, midi_from_muspy
from train import VAETrainer

Set global seed:

In [ ]:
seed = 42
set_seed(seed)

Load model:

In [ ]:
models_dir = 'models/'
model = 'LMD2'
gpu = True
device_idx = 3

checkpoint = torch.load(os.path.join(models_dir, model, 'checkpoint'), map_location='cpu')
state_dict = checkpoint['model_state_dict']
params = torch.load(os.path.join(models_dir, model, 'params'), map_location='cpu')

In [ ]:
torch.cuda.set_device(device_idx)

device = torch.device("cuda") if gpu else torch.device("cpu")
print("Device:", device)
print("Device idx:", torch.cuda.current_device())

In [ ]:
params

In [ ]:
vae = VAE(**params['model'], device=device).to(device)
vae.load_state_dict(state_dict)
vae.eval()

In [ ]:
def generate_music(vae, z, s_cond=None, s_tensor_cond=None):
    
    # Get structure and content logits
    with torch.cuda.amp.autocast():
        _, c_logits, s_tensor_out = vae.decoder(z, s_cond)
    
    s_tensor = s_tensor_cond if s_tensor_cond != None else s_tensor_out
    
    # Build (n_batches x n_bars x n_tracks x n_timesteps x Sigma x d_token)
    # multitrack pianoroll tensor containing logits for each activation and
    # hard silences elsewhere
    mtp = dense_from_sparse_torch(c_logits, s_tensor)
    
    # Collapse bars dimension
    mtp = mtp.permute(0, 2, 1, 3, 4, 5)
    size = (mtp.shape[0], mtp.shape[1], -1, mtp.shape[4], mtp.shape[5])
    mtp = mtp.reshape(*size)
    
    return mtp, s_tensor

In [ ]:
import matplotlib as mpl


def save(mtp, dir, s_tensor=None, track_data=None, n_loop=1):
    
    track_data = ([('Drums', -1), ('Bass', 34), ('Guitar', 1), ('Strings', 83)]
                  if track_data == None else track_data)
    
    # Clear matplotlib cache (this avoids formatting problems with first plot)
    plt.clf()

    # Iterate over the generated n-bar sequences
    for i in range(mtp.size(0)):
        
        # Create the directory if it does not exist
        save_dir = os.path.join(dir, str(i))
        os.makedirs(save_dir, exist_ok=True)

        print("Saving midi sequence " + str(i+1) + "...")
        
        # Generate muspy song from multitrack pianoroll, then midi from muspy
        # and save
        muspy_song = muspy_from_dense_torch(mtp[i], track_data, 
                                            params['model']['resolution'])
        midi_from_muspy(muspy_song, save_dir, name='music')
        
        # Plot the pianoroll associated to the sequence
        preset = 'full'
        with mpl.rc_context({'lines.linewidth': 4, 
                             'axes.linewidth': 4, 'font.size': 34}):
            plot_pianoroll(muspy_song, save_dir, name='pianoroll',
                           figsize=(20, 10), fformat='png', preset=preset)
        
        # Plot structure_tensor if present
        if s_tensor != None:
            s_curr = s_tensor[i]
            s_curr = s_curr.permute(1, 0, 2)
            s_curr = s_curr.reshape(s_curr.shape[0], -1)
            with mpl.rc_context({'lines.linewidth': 1, 
                                 'axes.linewidth': 1, 'font.size': 14}):
                plot_struct(s_curr.cpu(), name='structure', 
                            save_dir=save_dir, figsize=(12, 3))

        if n_loop > 1:
            # Generate extended sequence
            print("Saving extended (looped) midi sequence " + str(i+1) + "...")
            extended = mtp[i].repeat(1, n_loop, 1, 1)
            extended = muspy_from_dense_torch(extended, track_data, 
                                              params['model']['resolution'])
            midi_from_muspy(extended, save_dir, name='extended')
        
        print()

    print("Finished.")

In [ ]:
def generate_z(bs, d_model):
    shape = (bs, d_model)

    z_norm = torch.normal(
        torch.zeros(shape, device=device),
        torch.ones(shape, device=device)
    )
    
    return z_norm
    

In [ ]:
import json

structure = True
structure_file = 'structure2bars.json'

n_sequences = 5
n_loop = 4
d_model = params['model']['d']
n_bars = params['model']['n_bars']
n_tracks = params['model']['n_tracks']
n_timesteps = 4*params['model']['resolution']
dir = 'tmpmusic/'

bs = n_sequences
track_data = [('Drums', -1), ('Bass', 34), ('Guitar', 1), ('Strings', 83)]

s, s_tensor = None, None

if structure:
    
    # Load structure tensor from file
    with open(structure_file, 'r') as f:
        s_tensor = json.load(f)
    
    s_tensor = torch.tensor(s_tensor)
    s_tensor = s_tensor.bool()
    s_tensor = s_tensor.unsqueeze(0).repeat(bs, 1, 1, 1)
    s = vae.decoder._structure_from_binary(s_tensor)

z = generate_z(bs, d_model)
mtp, s_tensor = generate_music(vae, z, s, s_tensor)
save(mtp, dir, s_tensor, track_data, n_loop)

In [ ]:
import torch
import json

# 1. Create a Tensor
tensor = torch.zeros(2, 4, 32)  # Replace with your actual tensor.
tensor[:, :, ::8] = 1
tensor = tensor.int()
# 2. Convert Tensor to List
tensor_list = tensor.tolist()

# 3. Write to JSON File
with open('structure2bars.json', 'w') as f:
    json.dump(tensor_list, f, indent=4)


In [ ]:
from utils import plot_structure

s = torch.zeros(2, 4, 32)

plot_structure()

In [ ]:
def get_track_edges(s_tensor, edge_type_idx=0):

    a_t = s_tensor.t()
    idxs = torch.stack(torch.where(a_t == 1)).t()

    # Create node labels
    labels = torch.zeros(s_tensor.size())
    s_tensor_idxs = torch.where(s_tensor == 1)
    num_nodes = len(s_tensor_idxs[0])
    labels[s_tensor_idxs] = torch.arange(num_nodes, dtype=torch.float)
    labels = labels.t()

    track_edges = []

    for track in range(a_t.size(1)):
        tr_idxs = list(idxs[idxs[:, 1] == track])
        e_idxs = [(tr_idxs[i],
                   tr_idxs[i+1]) for i in range(len(tr_idxs)-1)]
        edges = [(labels[tuple(e[0])], labels[tuple(e[1])],
                  edge_type_idx+track, e[1][0]-e[0][0]) for e in e_idxs]
        inv_edges = [(e[1], e[0], *e[2:]) for e in edges]
        track_edges.extend(edges)
        track_edges.extend(inv_edges)

    return torch.tensor(track_edges, dtype=torch.long)

In [ ]:
def get_track_edges(s_tensor, edge_type_idx=0):

    a_t = s_tensor.t()
    idxs = torch.stack(torch.where(a_t == 1)).t()

    # Create node labels
    labels = torch.zeros(s_tensor.size())
    s_tensor_idxs = torch.where(s_tensor == 1)
    num_nodes = len(s_tensor_idxs[0])
    labels[s_tensor_idxs] = torch.arange(num_nodes, dtype=torch.float)
    labels = labels.t()

    track_edges = []

    for track in range(a_t.size(1)):
        tr_idxs = list(idxs[idxs[:, 1] == track])
        e_idxs = [(tr_idxs[i],
                   tr_idxs[i+1]) for i in range(len(tr_idxs)-1)]
        edges = [(labels[tuple(e[0])], labels[tuple(e[1])],
                  edge_type_idx+track, e[1][0]-e[0][0]) for e in e_idxs]
        inv_edges = [(e[1], e[0], *e[2:]) for e in edges]
        track_edges.extend(edges)
        track_edges.extend(inv_edges)

    return torch.tensor(track_edges, dtype=torch.long)

In [ ]:
import torch
a = torch.tensor([[1, 0, 0, 0], [1, 0, 1, 0], [0, 0, 1, 1], [1, 0, 1, 1]])
a

In [ ]:
ones_idxs = torch.nonzero(a, as_tuple=True)

ones_idxs[1][ones_idxs[0] == 3]

In [ ]:
c = ones_idxs[ones_idxs[:, 0] == 3]
for i, j in zip(c[:-1], c[1:]):
    print(i, j)
    print(a[i])

In [ ]:
torch.tensor([[3, 1], [9, 10]])[torch.tensor([1, 0])]

In [ ]:
a = torch.tensor([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]])

In [ ]:
import torch

edge_type_idx = 0
track_edges = []

# Indices where the binary structure tensor is active
ones_idxs = torch.nonzero(a, as_tuple=True)

# Build a node label tensor which has node labels in place of each activation 
# in the stucture tensor
labels = torch.zeros_like(a)
n_nodes = len(ones_idxs[0])
labels[ones_idxs] = torch.arange(n_nodes)

# For each track, add direct and inverse edges between consecutive nodes
for track in range(a.shape[0]):
    # List of active timesteps in the current track
    ts = list(ones_idxs[1][ones_idxs[0] == track])
    edges = [
        # Edge tuple: (u, v, type, ts_distance). Zip is used to obtain 
        # consecutive active timesteps. Edges in different tracks have different
        # types.
        (labels[track, t1], labels[track, t2], edge_type_idx + track, t2 - t1)
        for t1, t2 in zip(ts[:-1], ts[1:])
    ]
    inverse_edges = [(u, v, t, d) for (v, u, t, d) in edges]
    track_edges.extend(edges + inverse_edges)

torch.tensor(track_edges, dtype=torch.long)

In [ ]:
s_tensor = torch.tensor([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0]])

In [ ]:
import itertools

onset_edges = []
edge_type = 5

# Indices where the binary structure tensor is active
ones_idxs = torch.nonzero(s_tensor, as_tuple=True)

# Build a node label tensor which has node labels in place of each activation 
# in the stucture tensor
labels = torch.zeros_like(s_tensor)
n_nodes = len(ones_idxs[0])
labels[ones_idxs] = torch.arange(n_nodes)

# Add direct and inverse edges between nodes played in the same timestep
for ts in range(s_tensor.shape[1]):
    # List of active tracks in the current timestep
    tracks = list(ones_idxs[0][ones_idxs[1] == ts])
    # Obtain all possible pairwise combinations of active tracks
    combinations = list(itertools.combinations(tracks, 2))
    edges = [
        # Edge tuple: (u, v, type, ts_distance(=0)).
        (labels[track1, ts], labels[track2, ts], edge_type, 0)
        for track1, track2 in combinations
    ]
    inverse_edges = [(u, v, t, d) for (v, u, t, d) in edges]
    onset_edges.extend(edges + inverse_edges)

torch.tensor(onset_edges, dtype=torch.long)

In [ ]:
def get_next_edges_torch(s_tensor, edge_type_ind=5):

    # Indices where the binary structure tensor is active
    ones_idxs = torch.nonzero(s_tensor, as_tuple=True)
    #print(ones_idxs)
    # List of active timesteps
    tss = torch.nonzero(torch.any(s_tensor.bool(), dim=0)).squeeze()
    print(tss)
    #ts_acts = torch.any(a_t, dim=1)
    #print(ts_acts)
    #ts_inds = torch.where(ts_acts)[0]
    #print(ts_inds)
    
    # Build a node label tensor which has node labels in place of each activation 
    # in the stucture tensor
    labels = torch.zeros_like(s_tensor, dtype=torch.long)
    n_nodes = len(ones_idxs[0])
    labels[ones_idxs] = torch.arange(n_nodes)

    next_edges = []

    for i in range(tss.size(0)-1):
        # Get consecutive active timesteps
        t1, t2 = tss[i], tss[i+1]
        # Get all the active tracks in the two timesteps
        t1_tracks = ones_idxs[0][ones_idxs[1] == t1]
        t2_tracks = ones_idxs[0][ones_idxs[1] == t2]
        
        # Combine the source and destination tracks, removing combinations with
        # the same source and destination track (since these represent track
        # edges).
        tracks_product = list(itertools.product(t1_tracks, t2_tracks))
        tracks_product = [(track1, track2) 
                          for (track1, track2) in tracks_product 
                          if track1 != track2]
        # Edge tuple: (u, v, type, ts_distance).
        edges = [(labels[track1, t1], labels[track2, t2], 7, t2 - t1) 
                 for track1, track2 in tracks_product]

        next_edges.extend(edges)

    return torch.tensor(next_edges, dtype=torch.long)

In [ ]:
def get_track_features(s_tensor):
    
    ones_idxs = torch.nonzero(s_tensor)
    n_nodes = len(ones_idxs)
    print(ones_idxs)
    print(n_nodes)
    tracks = ones_idxs[0]
    print(tracks)

    n_tracks = s_tensor.size(0)
    features = torch.zeros((n_nodes, n_tracks))
    
    features[torch.arange(n_nodes), tracks] = 1

    return features

In [ ]:
s_tensor = torch.tensor([[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1]])
print(s_tensor)
get_track_features(s_tensor.bool())

In [ ]:
get_track_edges(a)

In [1]:
from torch.utils.data import Subset
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader
from data import PolyphemusDataset

bs = 64
nw = 10
n_bars = 8

#bs = 32
#nw = 10
#n_bars = 16

ds_dir = "data/prova/"

dataset = PolyphemusDataset(ds_dir, n_bars)
ds_len = len(dataset)

print('Dataset len:', len(dataset))

train_len = int(0.7 * len(dataset)) 
valid_len = int(0.1 * len(dataset))
test_len = len(dataset) - train_len - valid_len
tr_set, vl_set, ts_set = random_split(dataset, (train_len, valid_len, test_len))

trainloader = DataLoader(tr_set, batch_size=bs, shuffle=True, num_workers=nw)
validloader = DataLoader(vl_set, batch_size=bs, shuffle=False, num_workers=nw)

tr_len = len(tr_set)
vl_len = len(vl_set)
ts_len = len(ts_set)

print('TR set len:', len(tr_set))
print('VL set len:', len(vl_set))
print('TS set len:', len(ts_set))

Dataset len: 2098
TR set len: 1468
VL set len: 209
TS set len: 421


In [ ]:
for i, g in enumerate(trainloader):
    print(g)